In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

class Perceptron:
    def __init__(self, input_size, learning_rate=0.1, epochs=10):
        """
        Инициализация параметров персептрона.

        :param input_size: количество входных признаков
        :param learning_rate: скорость обучения (alpha)
        :param epochs: количество эпох обучения
        """
        # Инициализация весов
        self.weights = np.random.randn(input_size + 1)  # Добавляем bias
        self.lr = learning_rate
        self.epochs = epochs

    def activation_function(self, x):
        """
        Функция активации — сигмоид.
        Преобразует входное значение в диапазон (0,1).

        :param x: входное значение (сумма взвешенных входов)
        :return: выходное значение после применения сигмоидной функции
        """
        """
        Из-за ошибки RuntimeWarning: overflow encountered in exp return 1 / (1 + np.exp(-x))  # Сигмоидная функция
        связаной с тем, что значения, которые попадают в экспоненциальную функцию np.exp(-x), 
        могут быть слишком большими или слишком маленькими, что приводит к переполнению (overflow). 
        Это может происходить, если сумма взвешенных входов (линейная комбинация) слишком велика по абсолютной величине.
        """
        # Ограничиваем значения x, чтобы избежать переполнения
        x = np.clip(x, -700, 700)  # Значения для функции exp не должны быть слишком большими
        return 1 / (1 + np.exp(-x))  # Сигмоидная функция

    def predict(self, X):
        """
        Вычисляет предсказания модели.

        :param X: входные данные (матрица признаков)
        :return: массив предсказанных значений (вероятности)
        """
        # Добавляем столбец из единиц к X для учета bias
        X = np.insert(X, 0, 1, axis=1)
        # Вычисляем сумму взвешенных входов и применяем активационную функцию
        return self.activation_function(np.dot(X, self.weights))

    def train(self, X, y, method='gradient_descent'):
        """
        Обучает модель методом градиентного спуска или методом подкрепления.

        :param X: матрица признаков (размер N x M)
        :param y: вектор целевых значений (размер N)
        :param method: метод обучения ('gradient_descent' или 'reinforcement')
        """
        
        # Добавляем столбец из единиц к X для учета bias
        X = np.insert(X, 0, 1, axis=1)

        for epoch in range(self.epochs):  # Цикл по эпохам
            for i in range(X.shape[0]):  # Проход по каждому объекту
                # Вычисляем линейную комбинацию входов и весов
                z = np.dot(X[i], self.weights)
                # Применяем функцию активации
                y_pred = self.activation_function(z)
                # Вычисляем ошибку
                error = y[i] - y_pred

                if method == 'gradient_descent':
                    # Градиентный спуск: обновляем веса с учетом ошибки
                    self.weights += self.lr * error * X[i]
                
                elif method == 'reinforcement':
                    # Метод подкрепления: обновление в зависимости от ошибки
                    if error > 0:
                        # Положительное подкрепление: увеличиваем веса
                        self.weights += self.lr * X[i]
                    elif error < 0:
                        # Отрицательное подкрепление: уменьшаем веса
                        self.weights -= self.lr * X[i]

                else:
                    raise ValueError("Метод обучения не поддерживается. Выберите 'gradient_descent' или 'reinforcement'.")

    def evaluate(self, X, y):
        """
        Оценивает точность модели.

        :param X: тестовые входные данные
        :param y: реальные метки классов
        :return: точность классификации
        """
        # Получаем предсказания модели
        predictions = self.predict(X)
        # Преобразуем вероятности в бинарные классы (0 или 1)
        predictions = (predictions >= 0.5).astype(int)
        # Сравниваем предсказания с реальными метками
        accuracy = np.mean(predictions == y)
        return accuracy

# Загрузка данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Преобразуем изображения в одномерные векторы (28x28 = 784)
x_train = x_train.reshape(x_train.shape[0], -1).astype(np.float32)
x_test = x_test.reshape(x_test.shape[0], -1).astype(np.float32)

# Нормализуем данные
x_train /= 255.0
x_test /= 255.0

# Преобразуем метки в бинарные (для двух классов, например 0 и 1)
y_train = (y_train == 0).astype(int)  # Пример для классификации только 0 и 1
y_test = (y_test == 0).astype(int)

# Создание и обучение персептрона с методом градиентного спуска
perceptron = Perceptron(input_size=784, learning_rate=0.1, epochs=10)
perceptron.train(x_train, y_train, method='gradient_descent')

# Оценка точности на тестовых данных после обучения методом градиентного спуска
accuracy = perceptron.evaluate(x_test, y_test)
print(f"Точность на тестовых данных (метод градиентного спуска): {accuracy:.4f}")

# Создание и обучение персептрона с методом подкрепления
perceptron = Perceptron(input_size=784, learning_rate=0.1, epochs=10)
perceptron.train(x_train, y_train, method='reinforcement')

# Оценка точности на тестовых данных после обучения методом подкрепления
accuracy = perceptron.evaluate(x_test, y_test)
print(f"Точность на тестовых данных (метод подкрепления): {accuracy:.4f}")

11501568/11490434 [==============================] - 2s 0us/step
Точность на тестовых данных (метод градиентного спуска): 0.9920
Точность на тестовых данных (метод подкрепления): 0.9020
